In [0]:
df = spark.sql("""
    with cte as (
        select 
            location,year,month,
            round(avg(case when policy_school_close = -1 then null else policy_school_close end), 2) as avg_school_close,    
            round(avg(case when policy_work_close = -1 then null else policy_work_close end), 2) as avg_work_close,     
            round(avg(case when policy_stay_home = -1 then null else policy_stay_home end), 2) as avg_stay_home,   
            round(avg(case when policy_travel_control = -1 then null else policy_travel_control end), 2) as avg_travel_control,     
            round(avg(case when policy_test_policy = -1 then null else policy_test_policy end), 2) as avg_test_policy, 
            round(avg(case when policy_trace = -1 then null else policy_trace end), 2) as avg_trace,
            round(avg(case when policy_stringency = 0 then null else policy_stringency end), 2) as avg_stringency_index 
        FROM global_temp.covidcombined
        GROUP BY location, year, month
    )

    SELECT *,dense_rank() OVER (PARTITION BY location,year ORDER BY avg_stringency_index DESC) AS monthly_policy_strictness_rank,
        dense_rank() OVER (PARTITION BY location,year ORDER BY avg_travel_control DESC) AS monthly_travel_control_rank
    FROM cte
    ORDER BY location,year, monthly_policy_strictness_rank
""")


In [0]:
df_2 = df.fillna({
    "avg_school_close": -1,
    "avg_work_close": -1,
    "avg_stay_home": -1,
    "avg_travel_control": -1,
    "avg_test_policy": -1,
    "avg_trace": -1,
    "avg_stringency_index": -1
})

In [0]:
df_2.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/monthly_policy_summary")